# Full energy optimization demo

This notebook shows how to generate synthetic inputs for the full energy optimization
problem (space heating, hot water, and EV charging) and plot the resulting consumption
profiles.


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Ensure repository code is importable
import sys
from pathlib import Path
repo_root = Path('..').resolve()
sys.path.append(str(repo_root / 'Codes' / 'sourcecode'))

from RC_Optimization import optimize_full_energy_system, build_tariff


## Build synthetic inputs

- Two days of 30-minute intervals using the helper tariff builder
- Daily outdoor temperatures and solar irradiance following smooth sinusoids
- Two heating set-point schedules to compare (comfort vs. flexible)
- Hot water demand in morning/evening bursts
- EV availability overnight with a target state of charge by the end of day two


In [ ]:
# Time axis and tariff
start = pd.Timestamp('2024-01-01')
n_days = 2
step = '30min'
tariff = build_tariff(start, n_days=n_days, step=step, type='cosy')

# Derived quantities
n_steps = len(tariff)
dt_seconds = (tariff.index[1] - tariff.index[0]).total_seconds()
hours = tariff.index.hour + tariff.index.minute / 60.0

# Weather profiles (smooth diurnal cycles)
Tout = 5 + 6 * np.sin(2 * np.pi * (hours - 8) / 24)  # outdoor temperature in °C
S = np.clip(550 * np.sin(np.pi * (hours - 6) / 12), 0, None)  # solar irradiance in W/m²

# Heating set-points
comfort_setpoint = np.where((hours >= 6) & (hours < 9) | (hours >= 17) & (hours < 22), 21.0, 17.0)
flex_setpoint = np.where((hours >= 6) & (hours < 8) | (hours >= 18) & (hours < 21), 20.0, 16.0)
setpoint_sequences = [comfort_setpoint, flex_setpoint]

# Hot water demand (W) peaks morning/evening
def hw_profile(hour_array):
    morning = 2500 * np.exp(-0.5 * ((hour_array - 7) / 1.0) ** 2)
    evening = 2500 * np.exp(-0.5 * ((hour_array - 19) / 1.0) ** 2)
    return morning + evening

hw_demand = hw_profile(hours)

# Base electric load (W)
base_electric = 400 * np.ones(n_steps)

# EV parameters
av_mask = ((hours < 7) | (hours >= 18)).astype(float)  # EV at home overnight

ev_params = {
    'ev_capacity': 60.0,  # kWh
    'ev_target': 30.0,    # require half charged by end of horizon
    'ev_charge_max': 7.0, # kW
    'ev_availability': av_mask,
}


## Run the optimization

We evaluate both set-point schedules in one call. The solver returns the optimal solution for each schedule and highlights the least-cost option via the `best_key` and `best_result` entries.


In [ ]:
# Thermal model parameters
params = {
    'R1': 1 / 200,  # K/W
    'C1': 3e7,      # J/K
    'g': 10.0,      # solar gain factor
    'dt': dt_seconds,
    'T0': 20.0,
    'tol': 1.0,
    'COP': 3.5,
    'etaB': 0.9,
    'Qhp_max': 5e3,   # W
    'Qbo_max': 20e3,  # W
}

results = optimize_full_energy_system(
    tariff=tariff,
    Tout=Tout,
    S=S,
    setpoint_sequences=setpoint_sequences,
    hw_demand=hw_demand,
    base_electric=base_electric,
    day_ahead=True,
    **params,
    **ev_params,
)

best_key = results.get('best_key', next(k for k in results if k.startswith('schedule_')))
best = results['best_result']['results'] if 'best_result' in results else results[best_key]['results']
best_cost = results['best_result']['cost'] if 'best_result' in results else results[best_key]['cost']
print(f"Best schedule: {best_key}, total cost = {best_cost:.2f}")

best.head()


## Plot consumption profiles

The figure shows:

- Electricity and gas prices
- Indoor temperature versus comfort band
- Space and hot water heat delivery
- EV state of charge
- Aggregate electricity and gas consumption profiles


In [ ]:
fig, axs = plt.subplots(5, 1, figsize=(12, 16), sharex=True)

axs[0].plot(tariff.index, tariff['elec_price'], label='Electricity price (p/kWh)')
axs[0].plot(tariff.index, tariff['gas_price'], label='Gas price (p/kWh)')
axs[0].legend(); axs[0].set_ylabel('Tariff'); axs[0].grid(True)

axs[1].plot(best.index, best['Tin'], label='Indoor temp')
axs[1].plot(best.index, best['T_set'], '--', label='Set-point')
axs[1].fill_between(best.index, best['T_low'], best['T_high'], color='lightblue', alpha=0.3, label='Comfort band')
axs[1].legend(); axs[1].set_ylabel('Temperature (°C)'); axs[1].grid(True)

axs[2].plot(best.index, best['Q_hp_space']/1000, label='HP space (kW)')
axs[2].plot(best.index, best['Q_bo_space']/1000, label='Boiler space (kW)')
axs[2].plot(best.index, best['Q_hp_hw']/1000, label='HP hot water (kW)')
axs[2].plot(best.index, best['Q_bo_hw']/1000, label='Boiler hot water (kW)')
axs[2].legend(); axs[2].set_ylabel('Thermal output (kW)'); axs[2].grid(True)

axs[3].plot(best.index, best['ev_soc'], label='EV SOC (kWh)')
axs[3].bar(best.index, best['P_ev_charge'], width=0.02, alpha=0.4, label='EV charge (kW)')
axs[3].legend(); axs[3].set_ylabel('EV metrics'); axs[3].grid(True)

# Aggregate consumption
heat_pump_elec = (best['Q_hp_space'] + best['Q_hp_hw']) / params['COP'] / 1000  # kW
other_elec = base_electric / 1000  # kW
ev_elec = best['P_ev_charge']  # already kW
gas_input = (best['Q_bo_space'] + best['Q_bo_hw']) / params['etaB'] / 1000  # kW (thermal -> fuel)

axs[4].plot(best.index, heat_pump_elec + other_elec + ev_elec, label='Electric load (kW)')
axs[4].plot(best.index, gas_input, label='Gas input (kW)')
axs[4].legend(); axs[4].set_ylabel('Power (kW)'); axs[4].grid(True)

plt.xlabel('Time')
plt.tight_layout()
plt.show()
